In [ ]:
import cv2
import numpy as np

# Lê o vídeo
video_path = 'surveillance.avi'
cap = cv2.VideoCapture(video_path)

# Obtém informações do vídeo
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_rate = 30

# Configura o gravador de vídeo
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('Video_With_Square.avi', fourcc, frame_rate, (frame_width, frame_height), 0)

# Inicializa parâmetros
alpha = 0.95
theta = 0.1

# Inicializa o modelo de fundo com o primeiro quadro
ret, frame = cap.read()
if not ret:
    raise ValueError("Erro ao ler o primeiro quadro do vídeo.")

background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)

# Dimensões do quadrado branco
square_size = 30

# Crie uma matriz quadrada branca de três canais
square = np.ones((square_size, square_size, 3), dtype=np.uint8) * 255

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte o quadro atual para escala de cinza
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)

    # Atualiza o modelo de fundo
    background = alpha * background + (1 - alpha) * gray_frame

    # Calcula a diferença entre o quadro atual e o fundo
    diff_img = np.abs(gray_frame - background)

    # Aplica um limiar para criar uma imagem binária
    thresh_img = (diff_img > theta).astype(np.uint8) * 255

    # Encontra o centro da imagem
    center_x, center_y = frame_width // 2, frame_height // 2

    # Define as coordenadas para desenhar o quadrado no centro do quadro
    x1, x2 = center_x - square_size // 2, center_x + square_size // 2
    y1, y2 = center_y - square_size // 2, center_y + square_size // 2

    # Desenha o quadrado branco no quadro
    frame[y1:y2, x1:x2] = square

    # Mostra os quadros em uma janela
    cv2.imshow('Video with Square', frame)

    # Grava o quadro atual no arquivo de saída
    out.write(frame)

    if cv2.waitKey(30) & 0xFF == 27:  # Pressione Esc para sair
        break

# Libera os recursos
cap.release()
out.release()
cv2.destroyAllWindows()
